In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import random
import glob
import pickle
import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from models import CNN_LSTM

In [12]:
# # Load a sample of your data
# sample_data = np.load('./data/PROCESSED/2021_51_101.npy')  # Replace 'sample_data.npy' with the path to your data file

# # Check the shape of the sample data
# print("Shape of sample data:", sample_data.shape)

# Load a sample of your data
sample_data = np.load('./temp_data/MODIS_LAND/2021_51_101.npy')  # Replace 'sample_data.npy' with the path to your data file

# Check the shape of the sample data
print("Shape of sample data:", sample_data.shape)

# Load a sample of your data
sample_data = np.load('./data/PROCESSED/2020_51_101.npy')  # Replace 'sample_data.npy' with the path to your data file

# Check the shape of the sample data
print("Shape of sample data:", sample_data.shape)


Shape of sample data: (89, 128, 266)
Shape of sample data: (9, 128, 38)


In [3]:
# Define generator function
def generator(IDs, yields, batch_size, cutoff=None):
    def load_data(ID):
        try:
            data = np.load('/data/PROCESSED_III/' + ID + '.npy')
            return data
        except Exception as e:
            # print('Error loading data:', e)
            return None

    while True:
        batch_features = np.zeros((batch_size, 38, 1, 128, 9)) if cutoff is None else np.zeros((batch_size, cutoff, 1, 128, 9))
        batch_yields = np.zeros(batch_size)
        
        for i in range(batch_size):
            index = random.choice(range(len(IDs)))
            ID = IDs[index]
            data = load_data(ID)
            
            if data is not None:
                if cutoff is not None:
                    if not np.isnan(data).any():
                        batch_features[i, :, :, :, :] = data[:cutoff, :, :, :]
                        batch_yields[i] = yields[ID]
                    else:
                        print('Data contains NaN values:', ID)
                else:
                    batch_features[i, :, :, :, :] = data
                    batch_yields[i] = yields[ID]
            # else:
            #     print('Failed to load data:', ID)
    
        yield torch.tensor(batch_features, dtype=torch.float32, device='cuda'), torch.tensor(batch_yields, dtype=torch.float32, device='cuda')

model_functions = {
    'CNN_LSTM': CNN_LSTM,
    # 'SepCNN_LSTM': SepCNN_LSTM,
    # 'CONVLSTM': CONVLSTM,
    # 'CONV3D': CONV3D,
    # 'CONVLSTM_CONV3D': CONVLSTM_CONV3D
}


print("CNN", CNN_LSTM)

# Datasets
yields = pickle.load(open('data/yields.p', 'rb'))

# Retrieve the first 500 entries from yields['train']
first_500_train = {key: yields['train'][key] for key in list(yields['train'])[:500]}

print(len(yields['train']), len(yields['validation']))

# Generators
training_generator = generator(list(yields['train'].keys()), yields['train'], 16)
validation_generator = generator(list(yields['validation'].keys()), yields['validation'], 16)

for model_name, model_function in model_functions.items():
    model = model_function(dimensions=[38, 1, 128, 9])
    model.to('cuda')
    optimizer = torch.optim.Adam(model.parameters())
    criterion = nn.MSELoss()
    # earlystop = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=5, verbose=1, mode='auto')
    # callbacks_list = [earlystop]
    
    for epoch in range(10):
        model.train()
        train_losses = []
        for batch_data, batch_labels in tqdm.tqdm(training_generator, desc=f"Epoch {epoch+1}/{100}"):
            optimizer.zero_grad()
            outputs = model(batch_data)
            loss = criterion(outputs, batch_labels.unsqueeze(1))
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
        
        model.eval()
        val_losses = []
        with torch.no_grad():
            for val_data, val_labels in validation_generator:
                val_outputs = model(val_data)
                val_loss = criterion(val_outputs, val_labels.unsqueeze(1))
                val_losses.append(val_loss.item())
        print(f"Epoch {epoch+1}/{100}, Train Loss: {np.mean(train_losses):.4f}, Val Loss: {np.mean(val_losses):.4f}")
        
        # earlystop(np.mean(val_losses), model)  # Check early stopping criterion
        
        # if earlystop.early_stop:
        #     print("Early stopping")
        #     break
    
    # Save the model
    torch.save(model, f'{model_name}.pt')


CNN <class 'models.CNN_LSTM'>
6550 977


Epoch 1/100: 0it [00:00, ?it/s]/home/kaywee/Documents/wee's things/cds-corn-prediction/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at ../aten/src/ATen/native/Convolution.cpp:1040.)
  return F.conv2d(input, weight, bias, self.stride,
Epoch 1/100: 31725it [14:22, 36.77it/s]


KeyboardInterrupt: 